In [15]:
import numpy as np
import pandas as pd

import konlpy
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


## 모델 성능 평가
1. 시맨틱 키워드 추출
2. 정답지 생성
3. 사용자 입력 텍스트 분석 -> 시맨틱 키워드 추출
4. 빈도수 비교
5. 평가

In [3]:
#파일 가져오기
df_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/최종_데이터셋6(월,일,색상,RGB).csv"

In [9]:
df = pd.read_csv(df_file) #, encoding='CP949'
df.drop(["일자", "선택한 색상", "중앙 색상들"], axis=1, inplace=True)
df.head(5)

,꽃,월,계절,꽃말,설명,이미지,색상
0,각시붓꽃,3,봄,"부끄러움, 세련됨",각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,보라
1,감국,10,가을,그윽한 향기,가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...,https://www.nihhs.go.kr/user/AttachFiles/FLOWF...,노랑
2,개나리,3,봄,"희망,깊은 정, 달성","우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...",https://file.honestflower.kr/media/images/ingr...,노랑
3,개나리,4,봄,희망,"개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...",https://file.honestflower.kr/media/images/ingr...,노랑
4,갯개미취,9,가을,추억,옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...,https://lh6.googleusercontent.com/proxy/yRZlN4...,보라


In [10]:
#nlp처리 할 데이터만 가져오기
df_nlp = df[["꽃", "꽃말", "설명", "색상"]]
print(df_nlp.shape)
df_nlp.head(5)

(832, 4)


,꽃,꽃말,설명,색상
0,각시붓꽃,"부끄러움, 세련됨",각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 여러가지 붓꽃들 중...,보라
1,감국,그윽한 향기,가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인 감국은 보는 ...,노랑
2,개나리,"희망,깊은 정, 달성","우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃, 개나리는 어렵지 않게...",노랑
3,개나리,희망,"개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리나리 개나리 잎...",노랑
4,갯개미취,추억,옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 일처럼 ...,보라


In [11]:
#꽃말/설명/색상 합치기
df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['색상'] + ". " + df_nlp['설명']
df_nlp.drop(["꽃말", "색상"], axis=1, inplace=True)
df_nlp.head(5)

C:\Users\pc\AppData\Local\Temp\ipykernel_15044\1572906366.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['색상'] + ". " + df_nlp['설명']
C:\Users\pc\AppData\Local\Temp\ipykernel_15044\1572906366.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp.drop(["꽃말", "색상"], axis=1, inplace=True)


,꽃,설명
0,각시붓꽃,"부끄러움, 세련됨. 보라. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그..."
1,감국,그윽한 향기. 노랑. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지...
2,개나리,"희망,깊은 정, 달성. 노랑. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표..."
3,개나리,"희망. 노랑. 개나리, 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. ‘나리..."
4,갯개미취,추억. 보라. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을...


In [12]:
#특수문자 및 띄어쓰기 제거
import re

def remove_special_characters(text):
    if not isinstance(text, str):
        text = str(text)   
        
    # 특수문자 제거
    pattern = r'[^\w\s\.]' #문자,공백문자,마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' #중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
df_nlp.head(10)

C:\Users\pc\AppData\Local\Temp\ipykernel_15044\379896977.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)


,꽃,설명
0,각시붓꽃,부끄러움 세련됨. 보라. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래...
1,감국,그윽한 향기. 노랑. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지...
2,개나리,희망깊은 정 달성. 노랑. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인...
3,개나리,희망. 노랑. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리...
4,갯개미취,추억. 보라. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을...
5,거베라,신비 수수께끼. 분홍. 거베라만큼 크고 화려한 꽃도 드물다. 부를 상징하기도 한다....
6,검은포플라,용기. 빨강. 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다. 그렇다고...
7,겹 공작초,첫눈에 반함. 보라. 자기가 가장 아름답다고 믿는 세 여신이 있었다. 제우스의 아내...
8,겹 과꽃,나의 사랑은 당신보다깊다. 보라. 괴테의 희곡 파우스트 에 등장하는 소녀 마가렛이 ...
9,겹 소국,청초. 분홍. 국화 품종 중 작은 화형을 가진 국화를 소국이라고 하며 스프레이 형태...


In [13]:
#트큰화 (불용어 제외) - 빈도수 파악 위해
okt = Okt()

word_list=[]
for i in df_nlp['설명']:
    word_tokens = okt.morphs(i) #형태소 분석

    for j in word_tokens:
            word_list.append(j)
            
word_list

['부끄러움',
 '세련됨',
 '.',
 '보라',
 '.',
 '각시',
 '라',
 '하면',
 '이제',
 '막',
 '시집',
 '온',
 '새색시',
 '를',
 '연상',
 '케',
 '한다',
 '.',
 '그',
 '래서',
 '인지',
 '여러가지',
 '붓꽃',
 '들',
 '중',
 '에서',
 '각',
 '시',
 '붓꽃',
 '은',
 '바라볼수록',
 '다소곳함과',
 '소박함이',
 '느껴지는',
 '꽃',
 '이다',
 '.',
 '길고',
 '곧게',
 '뻗은',
 '가는',
 '잎',
 '도',
 '한층',
 '분위기',
 '를',
 '더한다',
 '.',
 '가정',
 '정원',
 '이나',
 '공원',
 '자연',
 '학습장',
 '등',
 '햇볕',
 '만',
 '조금',
 '드는',
 '곳',
 '이',
 '면',
 '어디',
 '서든',
 '화단',
 '용',
 '으로',
 '이용',
 '할',
 '수',
 '있다',
 '.',
 '키',
 '가',
 '작아',
 '분화',
 '용이',
 '나',
 '분',
 '경',
 '용',
 '소재',
 '로',
 '도',
 '이용',
 '할',
 '수',
 '있을',
 '것',
 '으로',
 '보인다',
 '.',
 '뿌리는',
 '약용',
 '으로',
 '쓰인다',
 '.',
 '그윽',
 '한',
 '향기',
 '.',
 '노랑',
 '.',
 '가을',
 '산야',
 '는',
 '국화',
 '과',
 '식물',
 '들',
 '차지',
 '다',
 '.',
 '특히',
 '노란',
 '꽃',
 '으로',
 '향기',
 '까지',
 '일품',
 '인',
 '감국',
 '은',
 '보는',
 '것',
 '만으로도',
 '좋지만',
 '향',
 '을',
 '맡거나',
 '말려서',
 '차로',
 '마셔도',
 '일품',
 '이다',
 '.',
 '그래서',
 '감국',
 '차가',
 '많이',
 '개발',
 '되어',
 '있다',
 '.',
 '감국',
 '은',
 '구절초',


In [42]:
#빈도수 확인
frequent = Counter(word_list).most_common()
frequent[520:550]

[('...', 15),
 ('될', 15),
 ('실제', 15),
 ('고급', 15),
 ('첫사랑', 15),
 ('봉오리', 14),
 ('사', 14),
 ('약간', 14),
 ('점', 14),
 ('있어서', 14),
 ('야생', 14),
 ('다시', 14),
 ('단', 14),
 ('가면', 14),
 ('자라는', 14),
 ('활용', 14),
 ('산', 14),
 ('당', 14),
 ('빨간색', 14),
 ('좋아하는', 14),
 ('청결', 14),
 ('널리', 14),
 ('쓰이는', 14),
 ('약', 14),
 ('화류', 14),
 ('가치', 14),
 ('라인', 14),
 ('있으면', 14),
 ('있기', 14),
 ('력', 14)]

### 1. 시맨틱 키워드 추출
- 꽃의 특징: 색상, 계절 등
- 꽃의 상징성: 꽃이 상징하는 감정이나 의미 (예: 사랑, 행복, 감사, 우정 등)
- 사용자의 요구: 사용자가 자주 언급하는 속성이나 요구사항(예: 선물, 장식, 기념일, 관계 등)

In [74]:
# 시맨틱 키워드 추출
# 설명 칼럼의 빈도수를 살펴보며 키워드 선정
keywords = ['봄', '여름', '가을', '겨울', '빨강', '주황', '노랑', '초록', '파랑', '보라', '분홍', '갈색', '흰색', '화려', '화려한', '귀여운', '정열', '은은한', '우아한', '청초한', '사랑스러운', '신비', '밝은', '사랑', '존경', '행복', '애정', '감사', '우정', '희망', '영원', '기쁨', '용기', '만남', '선물', '연인', '연애', '여자친구', '가족', '기념일', '생일', '생신', '발렌타인데이', '성년', '로즈데이', '부케']
flower_keywords = {flower: [keyword for keyword in keywords if keyword in description]
                   for flower, description in zip(df_nlp['꽃'], df_nlp['설명'])}

In [75]:
#정답 데이터셋
flower_keywords

{'각시붓꽃': ['보라'],
 '감국': ['가을', '노랑'],
 '개나리': ['봄', '노랑', '희망'],
 '갯개미취': ['보라', '만남'],
 '거베라': ['분홍', '화려', '화려한', '신비', '사랑'],
 '검은포플라': ['빨강', '용기'],
 '겹 공작초': ['보라', '사랑'],
 '겹 과꽃': ['보라', '사랑'],
 '겹 소국': ['분홍'],
 '겹 수선화': ['봄', '노랑', '화려', '화려한', '은은한', '선물'],
 '겹 스토크': ['분홍', '사랑'],
 '겹 알스트로메리아': ['주황', '화려', '만남'],
 '겹 캄파눌라': ['여름', '보라'],
 '겹 프릴 리시안셔스': ['분홍', '사랑'],
 '겹 해바라기': ['주황', '밝은'],
 '겹벗꽃': ['보라', '사랑'],
 '겹복숭아 꽃': ['봄', '빨강', '분홍', '행복'],
 '겹설유화': ['봄', '겨울', '초록', '사랑'],
 '겹튤립': ['주황', '애정', '영원'],
 '겹헬레보러스': ['빨강', '화려', '청초한'],
 '계란 소국': ['노랑', '귀여운', '밝은'],
 '골담초': ['노랑'],
 '골드트렌드셋터 장미': ['노랑', '우정'],
 '골드핀치 장미': ['노랑', '우정'],
 '골든게이트 카네이션': ['노랑', '존경'],
 '골든로드': ['노랑'],
 '골든볼': ['노랑'],
 '골디 카네이션': ['주황', '존경'],
 '골렘 카네이션': ['분홍', '존경'],
 '공작초': ['봄', '겨울', '보라'],
 '과꽃': ['보라'],
 '괭이눈': ['노랑'],
 '괭이밥': ['초록', '기쁨'],
 '괭이밥(옥살리스)': ['노랑', '사랑', '애정'],
 '구절초': ['가을', '보라', '분홍', '흰색', '밝은'],
 '구즈마니아': ['겨울', '빨강'],
 '국화': ['가을', '흰색'],
 '군자란': ['겨울', '주황', '화려', '화려

In [76]:
# 사용자 입력 텍스트에서 시맨틱 키워드 추출
def extract_keywords(text):
    okt = Okt()
    tokens = okt.morphs(text)
    return [token for token in tokens if token in keywords]

user_input = "부모님 생신이여서 꽃을 선물하고 싶은데 화려한 꽃으로 축하하고 싶어"
user_keywords = extract_keywords(user_input)

In [77]:
user_keywords

['생신', '선물', '화려한']

In [78]:
# 키워드 빈도수 계산
def keyword_frequency(keywords, keyword_list):
    vectorizer = CountVectorizer(vocabulary=keywords)
    return vectorizer.transform([' '.join(keyword_list)]).toarray()

user_vector = keyword_frequency(keywords, user_keywords)

In [80]:
# 꽃의 키워드 벡터 계산
flower_vectors = {flower: keyword_frequency(keywords, kw_list)
                  for flower, kw_list in flower_keywords.items()}

In [83]:
print(user_vector)
flower_vectors

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
  0 0 0 0 0 1 0 0 0 0]]


{'각시붓꽃': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], dtype=int64),
 '감국': array([[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], dtype=int64),
 '개나리': array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], dtype=int64),
 '갯개미취': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], dtype=int64),
 '거베라': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], dtype=int64),
 '검은포플라': array([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [84]:
# 코사인 유사도 계산
similarities = {flower: cosine_similarity(user_vector, vector)[0][0]
                for flower, vector in flower_vectors.items()}

# 가장 유사한 꽃 추천
recommended_flower = max(similarities, key=similarities.get)
print(f"추천하는 꽃: {recommended_flower}")

추천하는 꽃: 겹 수선화


이런 식으로 키워드의 빈도수를 비교해서 정답지를 따로 만들기 </br>
근데 이러면 tf-idf 느낌인거 아니야? 오히려 이게 정답이라고 볼 수 있나? 유사한 의미도 제거된 것 같은데??? 정말 확실한 것들만 넣어야 하나